In [3]:
## Retrieval agumented generation

In [17]:
import os
from dotenv import load_dotenv
import streamlit as st
from llama_index.core.service_context import ServiceContext
# from llama_index.llms import OpenAI
from llama_index.core import VectorStoreIndex
from llama_index.core import SimpleDirectoryReader
from llama_index.graph_stores.nebula import NebulaGraphStore
from llama_index.core.llms import LLM
from llama_index.llms.openai import OpenAI
from llama_index.core.response.pprint_utils import pprint_response

load_dotenv()

True

In [6]:
os.environ['OPENAI_API_KEY']= os.getenv('OPEN_API_KEY')
documents= SimpleDirectoryReader("data").load_data()

In [9]:
index= VectorStoreIndex.from_documents(documents, show_progress=2)

/Users/marwahfaraj/.pyenv/versions/3.8.12/envs/lama_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating embeddings: 100%|██████████| 99/99 [00:01<00:00, 70.67it/s]


In [10]:
index

In [11]:
query_engin= index.as_query_engine()

In [18]:
resopnse= query_engin.query("What is Yolo")
pprint_response(resopnse)

Final Response: YOLO (You Only Look Once) is a variant of object
detectors that has rapidly evolved since its inception in 2015, with
the latest release being YOLO-v8 in January 2023. The YOLO variants
are known for their real-time and high-classification performance
based on efficient computational parameters. These variants are
designed to address the requirements of automated quality inspection
within industrial surface defect detection, emphasizing the need for
fast detection, high accuracy, and deployment on constrained edge
devices.
